In [356]:
import pandas as pd

Metadata = pd.read_csv('C:\\Users\\Steffi Grace\\24592774_LLM_ILabResearch\\Notebooks\\Datasets\\Filtered_MetaData.csv')

In [357]:

import re
def preprocess_metadata(metadata):
    # Clean text data (e.g., remove special characters, lowercase)
    cleaned_metadata = metadata.copy() 
    cleaned_metadata['description'] = cleaned_metadata['description'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
    cleaned_metadata['description'] = cleaned_metadata['description'].str.lower()
    cleaned_metadata['title'] = cleaned_metadata['title'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
    cleaned_metadata['title'] = cleaned_metadata['title'].str.lower()
    cleaned_metadata['summary'] = cleaned_metadata['summary'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
    cleaned_metadata['summary'] = cleaned_metadata['summary'].str.lower()

    
    # Tokenize text data (split into words)
    cleaned_metadata['description'] = cleaned_metadata['description'].str.split()        
    cleaned_metadata['title'] = cleaned_metadata['title'].str.split()        
    cleaned_metadata['summary'] = cleaned_metadata['summary'].str.split()

    # Handle missing values (e.g., fill with a placeholder or remove rows)
    cleaned_metadata.fillna('', inplace=True)
    
    return cleaned_metadata

# Preprocess metadata
Preprocess_Metadata = preprocess_metadata(Metadata)

print("Preprocessed metadata:")
print(Preprocess_Metadata.head())


Preprocessed metadata:
                                               title  \
0  [2023w7, global, electric, vehicle, market, sh...   
1         [connected, electric, autonomous, vehicle]   
2            [electric, vehicle, charging, stations]   
3  [nyserda, electric, vehicle, drive, clean, reb...   
4  [impact, of, uncoordinated, plugin, electric, ...   

                                         description  \
0         [global, electric, vehicle, market, share]   
1         [connected, electric, autonomous, vehicle]   
2            [electric, vehicle, charging, stations]   
3  [new, york, states, charge, ny, initiative, of...   
4  [impact, of, uncoordinated, plugin, electric, ...   

                                             summary  \
0         [global, electric, vehicle, market, share]   
1  [romanian, new, car, registration, in, 2023, v...   
2  [statewise, data, from, 2001, is, classified, ...   
3      [coronoavirus, covid19, data, updated, daily]   
4  [adding, public, dat

In [358]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd

def preprocess_text(text):
    # Replace 'nan' string with an empty string
    if text == 'nan':
        return ""
    if isinstance(text, list):
        # Convert list of strings to a single string
        text = ' '.join(text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove punctuation and non-alphabetic characters, and single character tokens
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens if token.isalpha() and len(token) > 1]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text


In [359]:
Preprocess_Metadata.head()


,title,description,summary,tags,dataset_url,available_formats
0,"[2023w7, global, electric, vehicle, market, sh...","[global, electric, vehicle, market, share]","[global, electric, vehicle, market, share]",makeover monday cars vehicles electric ev mark...,https://data.world/makeovermonday/2023w7,['xlsx']
1,"[connected, electric, autonomous, vehicle]","[connected, electric, autonomous, vehicle]","[romanian, new, car, registration, in, 2023, v...",autonomous,https://data.world/smartcolumbusos/650b7e59-af...,['csv']
2,"[electric, vehicle, charging, stations]","[electric, vehicle, charging, stations]","[statewise, data, from, 2001, is, classified, ...",electric vehicle environment energy,https://data.world/townofcary/electric-vehicle...,"['dbf', 'prj', 'json', 'shp', 'csv', 'shx']"
3,"[nyserda, electric, vehicle, drive, clean, reb...","[new, york, states, charge, ny, initiative, of...","[coronoavirus, covid19, data, updated, daily]",ev electric vehicle bev phev ghg drive clean r...,https://data.world/data-ny-gov/thd2-fu8y,['csv']
4,"[impact, of, uncoordinated, plugin, electric, ...","[impact, of, uncoordinated, plugin, electric, ...","[adding, public, datasets, related, to, corona...",battery consumption data energy hybrid midwest...,https://data.world/us-doe-gov/8ae7e117-313b-40...,['xlsx']


In [360]:
  # Apply preprocessing to text columns
for column in ['title', 'description', 'summary']:
    Preprocess_Metadata[column] = Preprocess_Metadata[column].apply(preprocess_text)

In [361]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Load pre-trained T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Assuming Preprocess_Metadata is the DataFrame containing the dataset
metadata = Preprocess_Metadata.copy()

def preprocess_text(text):
    if text == 'nan':
        return ""
    if isinstance(text, str):
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens if token.isalpha() and len(token) > 1]
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text
    else:
        return ""

for column in ['title', 'description', 'summary']:
    metadata[column] = metadata[column].apply(preprocess_text)

metadata_embeddings = []
for index, row in metadata.iterrows():
    text = row['title'] + ' ' + row['description'] + ' ' + row['summary']
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        output = model.generate(input_ids)
    embeddings = model.get_input_embeddings()(input_ids).mean(dim=1).squeeze().detach().numpy()
    

    metadata_embeddings.append(embeddings)

user_input = input("Enter your query: ")
input_ids = tokenizer.encode(user_input, return_tensors="pt", max_length=512, truncation=True)
with torch.no_grad():
    output = model.generate(input_ids)
user_embeddings = model.get_input_embeddings()(input_ids).mean(dim=1).squeeze().detach().numpy()


similarities = cosine_similarity([user_embeddings], metadata_embeddings)[0]

def adjust_score(score, title, description, summary):
    completeness_factor = 0
    if title:
        completeness_factor += 0.1
    if description:
        completeness_factor += 0.1
    if summary:
        completeness_factor += 0.1
    return score + completeness_factor

adjusted_similarities = []
for i, score in enumerate(similarities):
    title, description, summary = metadata.iloc[i]['title'], metadata.iloc[i]['description'], metadata.iloc[i]['summary']
    adjusted_score = adjust_score(score, title, description, summary)
    adjusted_similarities.append(adjusted_score)

# Calculate the 95th percentile threshold
percentile_threshold = np.percentile(adjusted_similarities, 98)

dataset_urls_with_scores = zip(metadata['dataset_url'], adjusted_similarities)
sorted_dataset_urls_with_scores = sorted(dataset_urls_with_scores, key=lambda x: x[1], reverse=True)

print("Relevant dataset URLs sorted by similarity score (descending order):")
print("---------------------------------")
print("        Search text              ")
print("---------------------------------")
print(user_input)
print("---------------------------------")
print("                                 ")
print("---------------------------------")


for counter, (dataset_url, similarity_score) in enumerate(sorted_dataset_urls_with_scores, start=1):
    if similarity_score >= percentile_threshold:
        print(f"Counter: {counter}")
        print(f"Similarity Score: {similarity_score}")
        print(f"Dataset URL: {dataset_url}")
        metadata_index = metadata.index[metadata['dataset_url'] == dataset_url].tolist()[0]
        print(f"Title: {metadata['title'].iloc[metadata_index]}")
        print(f"Description: {metadata['description'].iloc[metadata_index]}")
        print(f"Summary: {metadata['summary'].iloc[metadata_index]}")
        print("\n")
        total_output = counter


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Relevant dataset URLs sorted by similarity score (descending order):
---------------------------------
        Search text              
---------------------------------
Corona Virus
---------------------------------
                                 
---------------------------------
Counter: 1
Similarity Score: 0.7031020998954773
Dataset URL: https://data.world/markmarkoh/coronavirus-data
Title: coronavirus daily data
Description: coronoavirus data updated daily
Summary: coronoavirus data updated daily


Counter: 2
Similarity Score: 0.6646250367164612
Dataset URL: https://data.world/chandrasekar/coronatracking
Title: coronatracking
Description: adding public datasets related corona virus update
Summary: adding public datasets related corona virus update




In [365]:
print(len(metadata))
print(len(similarities))

79
79


In [366]:
# Filter the labeled metadata with label == 1
New_Metadata = pd.read_csv('C:\\Users\\Steffi Grace\\24592774_LLM_ILabResearch\\Notebooks\\Datasets\\New_Prep_Metadata.csv')


In [367]:
def preprocess_metadata(metadata):
    # Clean text data (e.g., remove special characters, lowercase)
    cleaned_metadata = New_Metadata.copy() 
    cleaned_metadata['description'] = cleaned_metadata['description'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
    cleaned_metadata['description'] = cleaned_metadata['description'].str.lower()
    cleaned_metadata['title'] = cleaned_metadata['title'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
    cleaned_metadata['title'] = cleaned_metadata['title'].str.lower()
    cleaned_metadata['summary'] = cleaned_metadata['summary'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
    cleaned_metadata['summary'] = cleaned_metadata['summary'].str.lower()

    
    # Tokenize text data (split into words)
    cleaned_metadata['description'] = cleaned_metadata['description'].str.split()        
    cleaned_metadata['title'] = cleaned_metadata['title'].str.split()        
    cleaned_metadata['summary'] = cleaned_metadata['summary'].str.split()

    # Handle missing values (e.g., fill with a placeholder or remove rows)
    cleaned_metadata.fillna('', inplace=True)
    
    return cleaned_metadata

# Preprocess metadata
Preprocess_Metadata = preprocess_metadata(Metadata)

print("Preprocessed metadata:")
print(Preprocess_Metadata.head())


Preprocessed metadata:
        owner                                    id  \
0         hdx  80ded4e2-aa4f-42bc-ad08-6c1effc24123   
1         hdx  f92954dc-3f5b-407a-80a9-1e178280b0d7   
2  us-hhs-gov  87c842cf-095a-412e-8cf2-5d87ed07e34b   
3   alexandra                    food-related-words   
4        chhs  0114f5bb-4975-419d-95d9-5f0179a8de06   

                                               title  \
0  [europe, coronavirus, covid19, subnational, ca...   
1  [the, new, york, times, coronavirus, covid19, ...   
2  [provisional, death, counts, for, coronavirus,...   
3                             [food, related, words]   
4                              [food, affordability]   

                                         description  \
0  [europe, coronavirus, covid19, subnational, ca...   
1  [the, new, york, times, coronavirus, covid19, ...   
2  [provisional, death, counts, for, coronavirus,...   
3  [7000, words, that, relate, to, food, producti...   
4  [this, table, contains, da

In [368]:
Filtered_cols = ['title','description','summary','tags','dataset_url','available_formats']
New_Metadata = Preprocess_Metadata[Filtered_cols]

In [369]:
New_Metadata.head()


,title,description,summary,tags,dataset_url,available_formats
0,"[europe, coronavirus, covid19, subnational, ca...","[europe, coronavirus, covid19, subnational, ca...","[this, dataset, contains, the, number, of, con...","['affected', 'population', 'epidemics', 'and',...",https://data.world/hdx/80ded4e2-aa4f-42bc-ad08...,['csv']
1,"[the, new, york, times, coronavirus, covid19, ...","[the, new, york, times, coronavirus, covid19, ...","[original, title, the, new, york, times, coron...","['affected', 'population', 'epidemics', 'and',...",https://data.world/hdx/f92954dc-3f5b-407a-80a9...,['csv']
2,"[provisional, death, counts, for, coronavirus,...","[provisional, death, counts, for, coronavirus,...","[original, title, provisional, death, counts, ...","['coronavirus', 'deaths', 'ethnicity', 'mortal...",https://data.world/us-hhs-gov/87c842cf-095a-41...,['csv']
3,"[food, related, words]","[7000, words, that, relate, to, food, producti...","[manually, checked, only, once, so, not, entir...","['food', 'words', 'dictionary']",https://data.world/alexandra/food-related-words,['csv']
4,"[food, affordability]","[this, table, contains, data, on, the, average...","[this, table, contains, data, on, the, average...","['food', 'affordability', 'food', 'security', ...",https://data.world/chhs/0114f5bb-4975-419d-95d...,"['xlsx', 'json', 'pdf']"


In [370]:
# Assuming new_data contains the new dataset to be added
new_data_embeddings = []
for index, row in New_Metadata.iterrows():
    # Join the elements of the lists into a single string
    text = ' '.join(row['title']) + ' ' + ' '.join(row['description']) + ' ' + ' '.join(row['summary'])
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        output = model.generate(input_ids)
    embeddings = model.get_input_embeddings()(input_ids).mean(dim=1).squeeze().detach().numpy()
    new_data_embeddings.append(embeddings)

# Append new data embeddings to existing metadata embeddings
#metadata_embeddings.extend(new_data_embeddings)
metadata = metadata.append(New_Metadata, ignore_index=True)
index_start = len(metadata_embeddings)
metadata_embeddings[index_start:] = new_data_embeddings

# Compute similarities with the extended metadata embeddings
similarities = cosine_similarity([user_embeddings], metadata_embeddings)[0]



C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
C:\Users\Steffi Grace\AppData\Local\Temp\ipykernel_28004\2178811352.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata = metadata.append(New_Metadata, ignore_index=True)


In [371]:
print(len(metadata))
print(len(New_Metadata))
print(len(similarities))
print(len(metadata_embeddings))


84
5
84
84


In [372]:
user_input = input("Enter your query: ")
input_ids = tokenizer.encode(user_input, return_tensors="pt", max_length=512, truncation=True)
with torch.no_grad():
    output = model.generate(input_ids)
user_embeddings = model.get_input_embeddings()(input_ids).mean(dim=1).squeeze().detach().numpy()


similarities = cosine_similarity([user_embeddings], metadata_embeddings)[0]

def adjust_score(score, title, description, summary):
    completeness_factor = 0
    if title:
        completeness_factor += 0.1
    if description:
        completeness_factor += 0.1
    if summary:
        completeness_factor += 0.1
    return score + completeness_factor

adjusted_similarities = []
for i, score in enumerate(similarities):
    title, description, summary = metadata.iloc[i]['title'], metadata.iloc[i]['description'], metadata.iloc[i]['summary']
    adjusted_score = adjust_score(score, title, description, summary)
    adjusted_similarities.append(adjusted_score)

# Calculate the 95th percentile threshold
percentile_threshold = np.percentile(adjusted_similarities, 98)

dataset_urls_with_scores = zip(metadata['dataset_url'], adjusted_similarities)
sorted_dataset_urls_with_scores = sorted(dataset_urls_with_scores, key=lambda x: x[1], reverse=True)

print("Relevant dataset URLs sorted by similarity score (descending order):")
print("---------------------------------")
print("        Search text              ")
print("---------------------------------")
print(user_input)
print("---------------------------------")
print("                                 ")
print("---------------------------------")


for counter, (dataset_url, similarity_score) in enumerate(sorted_dataset_urls_with_scores, start=1):
    if similarity_score >= percentile_threshold:
        print(f"Counter: {counter}")
        print(f"Similarity Score: {similarity_score}")
        print(f"Dataset URL: {dataset_url}")
        metadata_index = metadata.index[metadata['dataset_url'] == dataset_url].tolist()[0]
        print(f"Title: {metadata['title'].iloc[metadata_index]}")
        print(f"Description: {metadata['description'].iloc[metadata_index]}")
        print(f"Summary: {metadata['summary'].iloc[metadata_index]}")
        print("\n")
        total_output = counter


Relevant dataset URLs sorted by similarity score (descending order):
---------------------------------
        Search text              
---------------------------------
Food related words
---------------------------------
                                 
---------------------------------
Counter: 1
Similarity Score: 0.8246365070343018
Dataset URL: https://data.world/alexandra/food-related-words
Title: ['food', 'related', 'words']
Description: ['7000', 'words', 'that', 'relate', 'to', 'food', 'production', 'of', 'food', 'cooking', 'ingredients', 'groceries', 'markets', 'etc', 'single', 'column', 'csv']
Summary: ['manually', 'checked', 'only', 'once', 'so', 'not', 'entirely', 'clean', 'as', 'it', 'contains', 'common', 'english', 'nouns', 'as', 'well', 'these', 'can', 'be', 'easily', 'sorted', 'out', 'however', 'some', 'words', 'are', 'indeed', 'food', 'related', 'such', 'as', 'brusselssprouts', 'or', 'whole', 'wheat']


Counter: 2
Similarity Score: 0.7313822984695435
Dataset URL: http

C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
